In [1]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

This is the notebook for the AI TTS Audiobook project.

In [2]:
! pip install -U pip
! pip install TTS

In [3]:
! pip install tensorboard

In [3]:
import os
from TTS.tts.configs.shared_configs import BaseDatasetConfig

In [3]:
import os
import datetime
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from tensorflow.keras.callbacks import TensorBoard

# Define output path
output_path = '/Users/benjaminlucas/Shaolin.AI Bootcamp/Homework/AI_TTS_Audiobook/Resources/LJSpeech-1.1'

# Define dataset config
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.csv",
    path=output_path
)

# Initialize audio processor
ap = AudioProcessor.init_from_config(config)

# Initialize model config with your specific configurations
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
)

# Initialize tokenizer from the config
tokenizer, config = TTSTokenizer.init_from_config(dataset_config)

# Load dataset samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# Initialize the model
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# Set up TensorBoard log directory
log_dir = os.path.join(output_path, 'logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Initialize trainer with TensorBoard callback
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples, callbacks=[tensorboard_callback]
)

# Start training
trainer.fit()

# Start TensorBoard
os.system(f'tensorboard --logdir={log_dir}')


NameError: name 'config' is not defined